In [57]:
import numpy as np
np.set_printoptions(linewidth = 200, precision = 4, suppress = True)

In [58]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [59]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [60]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [61]:
# One lower bound impl, and two upper bound impls
bounds[:, 60]

array([4.5265, 6.1343, 7.    ])

In [94]:
selected = np.where(nodes[:, 60] == 1)[0]
selected
current_i = len(selected)
selected

array([57, 82, 83, 84, 85])

In [95]:
np.argsort(-abs(data[85, :]))[:10]

array([85, 83, 84, 82, 79, 80, 12, 81, 19, 20])

In [96]:
# selected = np.r_[selected, [79]]
selected = np.r_[selected, [79, 100]]
new_i = len(selected)
selected

array([ 57,  82,  83,  84,  85,  79, 100])

In [97]:

new_i = len(selected)

In [98]:
data[selected, :][:, selected]

array([[ 1.    ,  0.4014,  0.3903,  0.3946,  0.3465,  0.4393,  0.248 ],
       [ 0.4014,  1.    ,  0.9643,  0.9311,  0.95  ,  0.8509, -0.2518],
       [ 0.3903,  0.9643,  1.    ,  0.9818,  0.9886,  0.9107, -0.2405],
       [ 0.3946,  0.9311,  0.9818,  1.    ,  0.9772,  0.9206, -0.2323],
       [ 0.3465,  0.95  ,  0.9886,  0.9772,  1.    ,  0.887 , -0.2399],
       [ 0.4393,  0.8509,  0.9107,  0.9206,  0.887 ,  1.    , -0.2105],
       [ 0.248 , -0.2518, -0.2405, -0.2323, -0.2399, -0.2105,  1.    ]])

In [99]:
# Use "raw" data SVD from here on out, singular values are slightly different.
np.random.seed(1)
n_obs = 100
raw_data = np.random.multivariate_normal(
    np.zeros_like(selected, float),
    data[selected, :][:, selected],
    size=n_obs)
raw_data.shape

(100, 7)

In [100]:
np.linalg.svd(raw_data)[1]

array([21.6917, 10.9232,  5.7372,  3.8253,  2.3708,  1.5045,  0.8171])

In [101]:
np.linalg.svd(np.c_[raw_data, raw_data])[1][::-1] / np.sqrt(2)

array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.8171,  1.5045,  2.3708,  3.8253,  5.7372, 10.9232, 21.6917])

In [102]:
v = np.linalg.svd(raw_data[:, 0:current_i])[2][0:1, :].T
v

array([[0.2381],
       [0.4818],
       [0.4924],
       [0.4816],
       [0.4866]])

In [105]:
import scipy.linalg
mat = scipy.linalg.block_diag(v @ v.T, np.zeros([new_i - current_i, new_i - current_i]), np.eye(current_i) - (v @ v.T), np.eye(new_i - current_i))
rotation_matrix_small = scipy.linalg.block_diag(
    np.linalg.svd(raw_data[:, 0:current_i])[2].T,
    np.eye(new_i - current_i))
# mat = mat @ scipy.linalg.block_diag(rotation_matrix_small, np.eye(6))

In [106]:
np.square(np.linalg.svd(np.c_[raw_data, raw_data] @ mat)[1][::-1])

array([  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.6677,   2.2636,   5.6205,  14.6327,  32.9158, 119.3171, 470.532 ])

In [107]:
# Gershgorin circle theorem after projecting onto two different matrices.
# We sparsified data_expanded, but Gershgorin bound is performing poorly!
apply_mat = np.c_[raw_data, raw_data] @ mat
data_expanded = apply_mat.T @ apply_mat
np.sort(abs(data_expanded).sum(axis=1))

array([  0.    ,   0.    ,  16.0809,  17.1679,  21.749 ,  32.5159, 155.4099, 220.4741, 251.7781, 478.7963, 509.2048, 509.4814, 514.5076, 520.7054])

In [108]:
# Gershgorin circle theorem (no change of variables).
np.sort(abs(raw_data.T @ raw_data).sum(axis=1))

array([204.8543, 320.3552, 468.8212, 507.5029, 510.7118, 513.8432, 521.3053])

In [109]:
abs(data_expanded).sum(axis=1)

array([251.7781, 509.4814, 520.7054, 509.2048, 514.5076,   0.    ,   0.    , 155.4099,  32.5159,  16.0809,  17.1679,  21.749 , 478.7963, 220.4741])

In [110]:
data_expanded

array([[ 22.2549,  45.0335,  46.0256,  45.009 ,  45.4777,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  40.1675,  -7.8099],
       [ 45.0335,  91.1267,  93.1343,  91.0772,  92.0257,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  81.2803, -15.8036],
       [ 46.0256,  93.1343,  95.186 ,  93.0837,  94.0531,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  83.071 , -16.1518],
       [ 45.009 ,  91.0772,  93.0837,  91.0278,  91.9758,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  81.2362, -15.7951],
       [ 45.4777,  92.0257,  94.0531,  91.9758,  92.9336,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  82.0822, -15.9596],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    

Actually decorrelate the vectors:

Use the left singular vector (representing all observations for some variable). Create 2 sets of variables which are not correlated. This is actually a lower bound on PCA. Maybe we can add one "linkage" on two off-diagonals (Hermitian) so that it is similar to the original matrix (connecting the projected variable and the residual). We can't actually break up a matrix into block diagonal in this way, eigenvalues do not work this way.

In [111]:
data_vector = np.linalg.svd(raw_data[:, 0:current_i])[0][:, 0:1]

In [112]:
data_proj = data_vector @ data_vector.T
data_proj_compl = np.eye(n_obs) - data_proj
raw_data_new = np.c_[data_proj @ raw_data, data_proj_compl @ raw_data]

In [113]:
np.square(np.linalg.svd(raw_data)[1]).max()

470.53199285265714

In [114]:
np.square(np.linalg.svd(raw_data_new)[1]).max()

467.76761546076415

In [115]:
np.square(np.linalg.svd(raw_data_new)[1])

array([467.7676, 119.4107,  33.0814,  17.0119,   5.7443,   2.2638,   0.6696,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ])

In [116]:
abs(raw_data_new.T @ raw_data_new).sum(axis=1)

array([251.7781, 509.4814, 520.7054, 509.2048, 514.5076, 454.4311,  88.3568, 155.4099,  32.5159,  16.0809,  17.1679,  21.749 ,  33.3968, 141.149 ])

In [117]:
raw_data_new.T @ raw_data_new

array([[ 22.2549,  45.0335,  46.0256,  45.009 ,  45.4777,  40.1675,  -7.8099,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 45.0335,  91.1267,  93.1343,  91.0772,  92.0257,  81.2803, -15.8036,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 46.0256,  93.1343,  95.186 ,  93.0837,  94.0531,  83.071 , -16.1518,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 45.009 ,  91.0772,  93.0837,  91.0278,  91.9758,  81.2362, -15.7951,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 45.4777,  92.0257,  94.0531,  91.9758,  92.9336,  82.0822, -15.9596,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 40.1675,  81.2803,  83.071 ,  81.2362,  82.0822,  72.4979, -14.096 ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ -7.8099, -15.8036, -16.1518, -15.7951, -15.9596, -14.096 ,   2.7407,   0.    ,   0.    ,   0.    

In [118]:
np.linalg.norm((raw_data_new.T @ raw_data_new)[0:new_i, 0:new_i], ord=2)

467.7676154607639

In [119]:

np.square(np.linalg.svd(raw_data_new[:, new_i:])[1])

array([119.4107,  33.0814,  17.0119,   5.7443,   2.2638,   0.6696,   0.    ])

In [120]:
raw_data_new.shape

(100, 14)

In [122]:
np.square(np.linalg.svd(raw_data_new[:, 0:new_i])[1]), np.linalg.svd(raw_data_new[:, 0:new_i])[1]

(array([467.7676,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ]),
 array([21.6279,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]))

In [124]:
np.square(np.linalg.svd(raw_data_new[:, new_i:])[1]), np.linalg.svd(raw_data_new[:, new_i:])[1]

(array([119.4107,  33.0814,  17.0119,   5.7443,   2.2638,   0.6696,   0.    ]),
 array([10.9275,  5.7516,  4.1246,  2.3967,  1.5046,  0.8183,  0.    ]))

In [125]:
np.sum(
    np.r_[
        np.square(np.linalg.svd(raw_data_new[:, 0:new_i])[1][0:1]),
        np.square(np.linalg.svd(raw_data_new[:, new_i:])[1])])

645.9493472102558

In [126]:
# np.square(
    # np.linalg.norm(
np.r_[
        np.linalg.svd(raw_data_new[:, 0:new_i])[1][0:1],
        np.linalg.svd(raw_data_new[:, new_i:])[1]]

array([21.6279, 10.9275,  5.7516,  4.1246,  2.3967,  1.5046,  0.8183,  0.    ])

Rank-One Update: We are trying to apply the first original left-singular vector projection onto the residual data.

w(lam) = 1 + rho sum(zeta ** 2 / (orig_eig_array - lam)) = 0

In [127]:
np.allclose(
    raw_data.T @ raw_data,
    (raw_data_new[:, 0:new_i].T @ raw_data_new[:, 0:new_i]) + (raw_data_new[:, new_i:].T @ raw_data_new[:, new_i:]))

True

In [140]:
np.linalg.eigvalsh(
    raw_data.T @ raw_data)

array([  0.6677,   2.2636,   5.6205,  14.6327,  32.9158, 119.3171, 470.532 ])

In [141]:
np.linalg.eigvalsh(
    raw_data_new[:, 0:new_i].T @ raw_data_new[:, 0:new_i])

array([ -0.    ,  -0.    ,  -0.    ,   0.    ,   0.    ,   0.    , 467.7676])

In [142]:
np.linalg.eigvalsh(
    raw_data_new[:, new_i:].T @ raw_data_new[:, new_i:])

array([  0.    ,   0.6696,   2.2638,   5.7443,  17.0119,  33.0814, 119.4107])

In [148]:
np.linalg.eigh(raw_data_new[:, 0:new_i].T @ raw_data_new[:, 0:new_i])[1][:, -1:].T @ \
    np.linalg.eigh(raw_data_new[:, new_i:].T @ raw_data_new[:, new_i:])[1]

array([[-0.9161,  0.0493,  0.0087,  0.1295, -0.3682,  0.0734,  0.0246]])

In [136]:
list(*np.linalg.svd(raw_data_new[:, 0:new_i])[0][:, 0:1].T @ np.linalg.svd(raw_data_new[:, new_i:], full_matrices=False)[0])

[5.551115123125783e-17,
 4.597017211338539e-16,
 -2.3240957769399273e-15,
 -1.1223660889569942e-15,
 -1.5785983631388945e-16,
 -2.2261923207644863e-15,
 0.9885052738229935]